In [1]:
import h2o
import numpy as np

import math
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, ShuffleSplit, learning_curve, GridSearchCV
from sklearn.metrics import recall_score, classification_report, confusion_matrix
import itertools

from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators import H2ORandomForestEstimator
from h2o.grid.grid_search import H2OGridSearch
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_121"; OpenJDK Runtime Environment (Zulu 8.20.0.5-linux64) (build 1.8.0_121-b15); OpenJDK 64-Bit Server VM (Zulu 8.20.0.5-linux64) (build 25.121-b15, mixed mode)
  Starting server from /home/rocket/anaconda2/h2o_jar/h2o.jar
  Ice root: /tmp/tmpS3f4d5
  JVM stdout: /tmp/tmpS3f4d5/h2o_rocket_started_from_python.out
  JVM stderr: /tmp/tmpS3f4d5/h2o_rocket_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,06 secs
H2O cluster version:,3.10.0.9
H2O cluster version age:,"1 year, 2 months and 29 days !!!"
H2O cluster name:,H2O_from_python_rocket_zc10lv
H2O cluster total nodes:,1
H2O cluster free memory:,3.464 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


# Learning Curve Definition

In [2]:
# Courtesy of http://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - An object to be used as a cross-validation generator.
          - An iterable yielding train/test splits.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : integer, optional
        Number of jobs to run in parallel (default 1).
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt


# Confusion Matrix Definition

In [3]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [4]:
df = h2o.import_file(path="../datasets/student_perform.csv")
df.describe()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Rows:480
Cols:17




,gender,NationalITy,PlaceofBirth,StageID,GradeID,SectionID,Topic,Semester,Relation,raisedhands,VisITedResources,AnnouncementsView,Discussion,ParentAnsweringSurvey,ParentschoolSatisfaction,StudentAbsenceDays,Class
type,enum,enum,enum,enum,enum,enum,enum,enum,enum,int,int,int,int,enum,enum,enum,enum
mins,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
mean,0.635416666667,NaN,NaN,NaN,NaN,NaN,NaN,0.489583333333,0.410416666667,46.775,54.7979166667,37.91875,43.2833333333,0.5625,0.608333333333,0.602083333333,NaN
maxs,1.0,13.0,13.0,2.0,9.0,2.0,11.0,1.0,1.0,100.0,99.0,98.0,99.0,1.0,1.0,1.0,2.0
sigma,0.481815279763,NaN,NaN,NaN,NaN,NaN,NaN,0.500413016613,0.492422575682,30.7792225827,33.0800066997,26.6112440819,27.6377350384,0.496595927951,0.488632078645,0.489978732159,NaN
zeros,175,9,9,33,147,283,59,245,283,9,7,10,0,210,188,191,142
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,15.0,16.0,2.0,20.0,Yes,Good,Under-7,M
1,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,20.0,20.0,3.0,25.0,Yes,Good,Under-7,M
2,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,10.0,7.0,0.0,30.0,No,Bad,Above-7,L


In [5]:
# Define predictors
y = df['Class']
X = df.drop('Class', axis=1)
X.describe()
y.describe()
# Apply the transformations to the data
X_train, X_test, y_train, y_test = train_test_split(X.as_data_frame().as_matrix(),y.as_data_frame().as_matrix())

Rows:480
Cols:16




,gender,NationalITy,PlaceofBirth,StageID,GradeID,SectionID,Topic,Semester,Relation,raisedhands,VisITedResources,AnnouncementsView,Discussion,ParentAnsweringSurvey,ParentschoolSatisfaction,StudentAbsenceDays
type,enum,enum,enum,enum,enum,enum,enum,enum,enum,int,int,int,int,enum,enum,enum
mins,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
mean,0.635416666667,NaN,NaN,NaN,NaN,NaN,NaN,0.489583333333,0.410416666667,46.775,54.7979166667,37.91875,43.2833333333,0.5625,0.608333333333,0.602083333333
maxs,1.0,13.0,13.0,2.0,9.0,2.0,11.0,1.0,1.0,100.0,99.0,98.0,99.0,1.0,1.0,1.0
sigma,0.481815279763,NaN,NaN,NaN,NaN,NaN,NaN,0.500413016613,0.492422575682,30.7792225827,33.0800066997,26.6112440819,27.6377350384,0.496595927951,0.488632078645,0.489978732159
zeros,175,9,9,33,147,283,59,245,283,9,7,10,0,210,188,191
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,15.0,16.0,2.0,20.0,Yes,Good,Under-7
1,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,20.0,20.0,3.0,25.0,Yes,Good,Under-7
2,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,10.0,7.0,0.0,30.0,No,Bad,Above-7


Rows:480
Cols:1




,Class
type,enum
mins,0.0
mean,NaN
maxs,2.0
sigma,NaN
zeros,142
missing,0
0,M
1,M
2,L


In [6]:
# Split for training
train, valid, test = df.split_frame(
    ratios = [0.6, 0.2],
    seed=4567,
    destination_frames=['train.hex','valid.hex','test.hex']
)

In [7]:
# Single decision tree, no boosting single tree
clf = H2ORandomForestEstimator(ntrees=1)
clf.train(x=X.columns, y=y.columns[0], training_frame=train)

# Show model summary
print clf

drf Model Build progress: |███████████████████████████████████████████████| 100%
Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1517517668925_1
Model Summary: 


,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,1.0,3.0,1563.0,7.0,10.0,8.666667,13.0,55.0,36.666668




ModelMetricsMultinomial: drf
** Reported on train data. **

MSE: 0.445833333333
RMSE: 0.6677075208
LogLoss: 14.7367422329
Mean Per-Class Error: 0.407407407407
Confusion Matrix: vertical: actual; across: predicted



H,L,M,Error,Rate
11.0,0.0,25.0,0.6944444,25 / 36
0.0,26.0,10.0,0.2777778,10 / 36
5.0,7.0,36.0,0.25,12 / 48
16.0,33.0,71.0,0.3916667,47 / 120


Top-3 Hit Ratios: 


k,hit_ratio
1,0.6083333
2,0.725
3,1.0


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error
,2018-02-01 15:41:23,0.019 sec,0.0,nan,nan,nan
,2018-02-01 15:41:23,0.390 sec,1.0,0.6677075,14.7367422,0.3916667


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
StudentAbsenceDays,41.0712051,1.0,0.3223420
raisedhands,27.5646954,0.6711441,0.2163379
NationalITy,9.6754503,0.2355775,0.0759365
AnnouncementsView,9.5429621,0.2323516,0.0748967
Discussion,7.3840170,0.1797857,0.0579525
VisITedResources,6.5579901,0.1596737,0.0514695
Topic,5.7524123,0.1400595,0.0451471
GradeID,5.1247411,0.1247770,0.0402209
Relation,3.9609463,0.0964410,0.0310870
ParentAnsweringSurvey,3.2629764,0.0794468,0.0256090


In [8]:
perf = clf.model_performance(valid)
perf


ModelMetricsMultinomial: drf
** Reported on test data. **

MSE: 0.456896551724
RMSE: 0.675941233928
LogLoss: 15.1416727962
Mean Per-Class Error: 0.371405567058
Confusion Matrix: vertical: actual; across: predicted



H,L,M,Error,Rate
13.0,1.0,14.0,0.5357143,15 / 28
1.0,18.0,4.0,0.2173913,5 / 23
7.0,6.0,23.0,0.3611111,13 / 36
21.0,25.0,41.0,0.3793103,33 / 87


Top-3 Hit Ratios: 


k,hit_ratio
1,0.6206896
2,0.7586207
3,1.0


In [9]:
y_pred = clf.predict(test).as_data_frame().as_matrix()
print y_pred
y_test = test['Rating'].as_data_frame().as_matrix()
X_test = test.as_data_frame().as_matrix()
# print y_test
# print X_test
cnf_matrix = confusion_matrix(y_test, y_pred)

plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['2', '3','4','5','6','7','8', '9', '10'], title='Confusion Matrix MLP Default')
plt.show()
plt.clf()

drf prediction progress: |████████████████████████████████████████████████| 100%
[['L' 0.0 1.0 0.0]
 ['H' 0.0 0.0 0.0]
 ['M' 0.0 0.0 1.0]
 ['L' 0.0 1.0 0.0]
 ['M' 0.0 0.0 1.0]
 ['M' 0.0 0.0 1.0]
 ['H' 1.0 0.0 0.0]
 ['M' 0.0 0.0 1.0]
 ['L' 0.0 1.0 0.0]
 ['L' 0.0 1.0 0.0]
 ['M' 0.0 0.0 1.0]
 ['L' 0.0 1.0 0.0]
 ['L' 0.0 0.0 0.0]
 ['L' 0.0 0.0 0.0]
 ['M' 0.0 0.0 1.0]
 ['M' 0.0 0.0 0.0]
 ['L' 0.0 1.0 0.0]
 ['L' 0.0 1.0 0.0]
 ['M' 0.5 0.0 0.5]
 ['L' 0.0 1.0 0.0]
 ['L' 0.0 1.0 0.0]
 ['L' 0.0 1.0 0.0]
 ['L' 0.0 1.0 0.0]
 ['M' 0.0 0.0 0.0]
 ['H' 0.0 0.0 0.0]
 ['L' 0.0 1.0 0.0]
 ['M' 0.0 0.0 1.0]
 ['M' 0.0 0.0 1.0]
 ['H' 0.5 0.0 0.5]
 ['L' 0.0 1.0 0.0]
 ['H' 1.0 0.0 0.0]
 ['M' 0.5 0.0 0.5]
 ['M' 0.5 0.0 0.5]
 ['H' 0.0 0.0 0.0]
 ['H' 0.0 0.0 0.0]
 ['M' 0.5 0.0 0.5]
 ['H' 1.0 0.0 0.0]
 ['M' 0.0 0.0 0.0]
 ['M' 0.0 0.0 1.0]
 ['M' 0.0 0.0 1.0]
 ['L' 0.0 1.0 0.0]
 ['L' 0.0 1.0 0.0]
 ['M' 0.5 0.0 0.5]
 ['M' 0.0 0.0 1.0]
 ['M' 0.0 0.0 1.0]
 ['L' 0.0 0.0 0.0]
 ['M' 0.0 0.0 0.0]
 ['M' 0.0 0.5 0.5]
 ['L' 0

H2OResponseError: Server error java.lang.IllegalArgumentException:
  Error: Column Rating not found
  Request: POST /99/Rapids
    data: {u'session_id': '_sid_bfa8', u'ast': '(tmp= py_8_sid_bfa8 (cols_py test.hex "Rating"))'}


In [10]:
# Lets see if we can do better on one tree
hyper_parameters = {'ntrees':[1], 
                    'max_depth':[20, 5], 
                    'balance_classes':[True, False], 
                    'distribution':['bernoulli', 'multinominal', 'gaussian', 'poisson', 'gamma', 'tweedie', 'laplace', 'huber'],
                    'histogram_type':['uniform_adaptive', 'random', 'quantiles_global', 'round_robin']}
grid_search = H2OGridSearch(H2ORandomForestEstimator, hyper_params=hyper_parameters)
grid_search.train(x=X.columns, y=y.columns[0], training_frame=train)
grid_search.show()

drf Grid Build progress: |████████████████████████████████████████████████| 100%
       balance_classes distribution   histogram_type max_depth ntrees  \
0                 true      poisson       RoundRobin         5      1   
1                 true        huber  QuantilesGlobal         5      1   
2                 true      laplace       RoundRobin         5      1   
3                false      poisson  UniformAdaptive         5      1   
4                 true        gamma       RoundRobin         5      1   
5                 true     gaussian  UniformAdaptive         5      1   
6                 true      poisson  UniformAdaptive         5      1   
7                 true     gaussian  QuantilesGlobal         5      1   
8                 true      tweedie           Random         5      1   
9                false      laplace       RoundRobin         5      1   
10                true        huber       RoundRobin         5      1   
11                true        gamma  Unifor

[112 rows x 8 columns]


In [11]:
# Now select the configuration with the lowest logloss
opt_tree = grid_search.get_grid(sort_by='rmse', decreasing=True)[-1]

In [12]:
opt_tree.model_performance(valid)


ModelMetricsMultinomial: drf
** Reported on test data. **

MSE: 0.287004834223
RMSE: 0.535728321282
LogLoss: 5.18644634466
Mean Per-Class Error: 0.401713825627
Confusion Matrix: vertical: actual; across: predicted



H,L,M,Error,Rate
9.0,0.0,19.0,0.6785714,19 / 28
3.0,16.0,4.0,0.3043478,7 / 23
3.0,5.0,28.0,0.2222222,8 / 36
15.0,21.0,51.0,0.3908046,34 / 87


Top-3 Hit Ratios: 


k,hit_ratio
1,0.6091954
2,0.8965517
3,1.0


In [11]:
opt_tree.hit_ratio_table()

AttributeError: type object 'H2ORandomForestEstimator' has no attribute 'hit_ratio_table'

In [10]:
h2o.shutdown()

H2O session _sid_bc7d closed.
